# Step 2: Model Building & Evaluation

In [1]:
# import the libraries
import h5py
import os
import csv
import pandas as pd
import numpy as np
import urllib
import glob
import pickle
import re
import keras

from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from sklearn import datasets
from imblearn.under_sampling import RandomUnderSampler
from keras.layers import Dense, Dropout, LSTM, Activation

print('Import complete successfully!')

Import complete successfully!


## Load feature data set

In [2]:
train_df10 = pd.read_csv('./data/02_preparedData/PdM_train_prepared_failure1.csv', sep=",", header=0)
train_df20 = pd.read_csv('./data/02_preparedData/PdM_train_prepared_failure2.csv', sep=",", header=0)
train_df30 = pd.read_csv('./data/02_preparedData/PdM_train_prepared_failure3.csv', sep=",", header=0)
train_df40 = pd.read_csv('./data/02_preparedData/PdM_train_prepared_failure4.csv', sep=",", header=0)
#train_df.head(10)

In [3]:
#balancing training data
#print('Balancing train data is in progress...')
# failure1
df_train10 = train_df10[["datetime","machineID","cycle","model","age","volt","rotate","pressure","vibration","RUL","label2","cycle_norm"]]
y_train10 = train_df10["label1"]
# failure2
df_train20 = train_df20[["datetime","machineID","cycle","model","age","volt","rotate","pressure","vibration","RUL","label2","cycle_norm"]]
y_train20 = train_df20["label1"]
# failure3
df_train30 = train_df30[["datetime","machineID","cycle","model","age","volt","rotate","pressure","vibration","RUL","label2","cycle_norm"]]
y_train30 = train_df30["label1"]
# failure4
df_train40 = train_df40[["datetime","machineID","cycle","model","age","volt","rotate","pressure","vibration","RUL","label2","cycle_norm"]]
y_train40 = train_df40["label1"]

rus10 = RandomUnderSampler()
rus20 = RandomUnderSampler()
rus30 = RandomUnderSampler()
rus40 = RandomUnderSampler()
#rus = RandomUnderSampler()
#rus = RandomUnderSampler(sampling_strategy='majority')
#rus = RandomUnderSampler(sampling_strategy={0:30000})

# failure1
df_resampled10, y_resampled10 = rus10.fit_resample(df_train10, y_train10)
df_balanced10 = pd.concat([df_resampled10, y_resampled10],axis=1)
train_df10 = df_balanced10.sort_values(['machineID', 'datetime'], ascending=[True, True])
train_df10 = train_df10[['datetime','machineID', 'cycle', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration', 'RUL', 'cycle_norm', 'label1', 'label2']]
train_df10.drop(train_df10.columns[[0]], axis=1, inplace=True)
# failure2
df_resampled20, y_resampled20 = rus20.fit_resample(df_train20, y_train20)
df_balanced20 = pd.concat([df_resampled20, y_resampled20],axis=1)
train_df20 = df_balanced20.sort_values(['machineID', 'datetime'], ascending=[True, True])
rain_df20 = train_df20[['datetime','machineID', 'cycle', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration', 'RUL', 'cycle_norm', 'label1', 'label2']]
train_df20.drop(train_df20.columns[[0]], axis=1, inplace=True)
# failure3
df_resampled30, y_resampled30 = rus30.fit_resample(df_train30, y_train30)
df_balanced30 = pd.concat([df_resampled30, y_resampled30],axis=1)
train_df30 = df_balanced30.sort_values(['machineID', 'datetime'], ascending=[True, True])
train_df30 = train_df30[['datetime','machineID', 'cycle', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration', 'RUL', 'cycle_norm', 'label1', 'label2']]
train_df30.drop(train_df30.columns[[0]], axis=1, inplace=True)
# failure4
df_resampled40, y_resampled40 = rus40.fit_resample(df_train40, y_train40)
df_balanced40 = pd.concat([df_resampled40, y_resampled40],axis=1)
train_df40 = df_balanced40.sort_values(['machineID', 'datetime'], ascending=[True, True])
train_df40 = train_df40[['datetime','machineID', 'cycle', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration', 'RUL', 'cycle_norm', 'label1', 'label2']]
train_df40.drop(train_df40.columns[[0]], axis=1, inplace=True)

#train_df.head(10)

print('Balancing train data completed successfully.')

Balancing train data completed successfully.


In [4]:
test_df10 = pd.read_csv('./data/02_preparedData/PdM_test_prepared_failure1.csv', sep=",", header=0)
test_df20 = pd.read_csv('./data/02_preparedData/PdM_test_prepared_failure2.csv', sep=",", header=0)
test_df30 = pd.read_csv('./data/02_preparedData/PdM_test_prepared_failure3.csv', sep=",", header=0)
test_df40 = pd.read_csv('./data/02_preparedData/PdM_test_prepared_failure4.csv', sep=",", header=0)

#test_df.head(10)

## Modelling

In [5]:
# pick a large window size of 50 cycles
sequence_length10 = 21
sequence_length20 = 21
sequence_length30 = 21
sequence_length40 = 21

In [6]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [7]:
# pick the feature columns 
# failure1
sequence_cols10 = ['cycle_norm']
key_cols10 = ['machineID', 'cycle']
label_cols10 = ['label1', 'label2', 'RUL']
input_features10 = test_df10.columns.values.tolist()
sensor_cols10 = [x for x in input_features10 if x not in set(key_cols10)]
sensor_cols10 = [x for x in sensor_cols10 if x not in set(label_cols10)]
sensor_cols10 = [x for x in sensor_cols10 if x not in set(sequence_cols10)]
# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols10.extend(sensor_cols10)

# failure2
sequence_cols20 = ['cycle_norm']
key_cols20 = ['machineID', 'cycle']
label_cols20 = ['label1', 'label2', 'RUL']
input_features20 = test_df20.columns.values.tolist()
sensor_cols20 = [x for x in input_features20 if x not in set(key_cols20)]
sensor_cols20 = [x for x in sensor_cols20 if x not in set(label_cols20)]
sensor_cols20 = [x for x in sensor_cols20 if x not in set(sequence_cols20)]
# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols20.extend(sensor_cols20)

# failure3
sequence_cols30 = ['cycle_norm']
key_cols30 = ['machineID', 'cycle']
label_cols30 = ['label1', 'label2', 'RUL']
input_features30 = test_df30.columns.values.tolist()
sensor_cols30 = [x for x in input_features30 if x not in set(key_cols30)]
sensor_cols30 = [x for x in sensor_cols30 if x not in set(label_cols30)]
sensor_cols30 = [x for x in sensor_cols30 if x not in set(sequence_cols30)]
# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols30.extend(sensor_cols30)

# failure4
sequence_cols40 = ['cycle_norm']
key_cols40 = ['machineID', 'cycle']
label_cols40 = ['label1', 'label2', 'RUL']
input_features40 = test_df40.columns.values.tolist()
sensor_cols40 = [x for x in input_features40 if x not in set(key_cols40)]
sensor_cols40 = [x for x in sensor_cols40 if x not in set(label_cols40)]
sensor_cols40 = [x for x in sensor_cols40 if x not in set(sequence_cols40)]
# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols40.extend(sensor_cols40)

#print(sensor_cols)
#print(sequence_cols)

In [8]:
# generator for the sequences
# failure1
seq_gen10 = (list(gen_sequence(train_df10[train_df10['machineID']==id10], sequence_length10, sequence_cols10)) 
           for id10 in train_df10['machineID'].unique())
# generate sequences and convert to numpy array
seq_array10 = np.concatenate(list(seq_gen10)).astype(np.float32)

# failure2
seq_gen20 = (list(gen_sequence(train_df20[train_df20['machineID']==id20], sequence_length20, sequence_cols20)) 
           for id20 in train_df20['machineID'].unique())
# generate sequences and convert to numpy array
seq_array20 = np.concatenate(list(seq_gen20)).astype(np.float32)

# failure3
seq_gen30 = (list(gen_sequence(train_df30[train_df30['machineID']==id30], sequence_length30, sequence_cols30)) 
           for id30 in train_df30['machineID'].unique())
# generate sequences and convert to numpy array
seq_array30 = np.concatenate(list(seq_gen30)).astype(np.float32)

# failure4
seq_gen40 = (list(gen_sequence(train_df40[train_df40['machineID']==id40], sequence_length40, sequence_cols40)) 
           for id40 in train_df40['machineID'].unique())
# generate sequences and convert to numpy array
seq_array40 = np.concatenate(list(seq_gen40)).astype(np.float32)

#seq_array.shape

In [9]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [10]:
# generate labels
# failure1
label_gen10 = [gen_labels(train_df10[train_df10['machineID']==id10], sequence_length10, ['label1']) 
             for id10 in train_df10['machineID'].unique()]
label_array10 = np.concatenate(label_gen10).astype(np.float32)

# failure2
label_gen20 = [gen_labels(train_df20[train_df20['machineID']==id20], sequence_length20, ['label1']) 
             for id20 in train_df20['machineID'].unique()]
label_array20 = np.concatenate(label_gen20).astype(np.float32)

# failure3
label_gen30 = [gen_labels(train_df30[train_df30['machineID']==id30], sequence_length30, ['label1']) 
             for id30 in train_df30['machineID'].unique()]
label_array30 = np.concatenate(label_gen30).astype(np.float32)

# failure4
label_gen40 = [gen_labels(train_df40[train_df40['machineID']==id40], sequence_length40, ['label1']) 
             for id40 in train_df40['machineID'].unique()]
label_array40 = np.concatenate(label_gen40).astype(np.float32)

#label_array.shape

In [ ]:
# if we need to check the network fitting input data
train_df10.to_csv('./data/train_df10.csv',index=False)
train_df20.to_csv('./data/train_df20.csv',index=False)
train_df30.to_csv('./data/train_df30.csv',index=False)
train_df40.to_csv('./data/train_df40.csv',index=False)

## LSTM Network

In [11]:
# build the network for failure1 data
# Feature weights
nb_features10 = seq_array10.shape[2]
nb_out10 = label_array10.shape[1]

# LSTM model
model10 = Sequential()

# The first layer
model10.add(LSTM(
         input_shape=(sequence_length10, nb_features10),
         units=400,
         return_sequences=True))

# Plus a 20% dropout rate
model10.add(Dropout(0.2))

# The second layer
model10.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model10.add(Dropout(0.2))

# The third layer
model10.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model10.add(Dropout(0.2))

# The fourth layer
model10.add(LSTM(
          units=200,
          return_sequences=False))

# Plus a 20% dropout rate
model10.add(Dropout(0.2))

# Dense sigmoid layer
model10.add(Dense(units=200, activation='sigmoid'))
model10.add(Dense(units=200, activation='sigmoid'))
model10.add(Dense(units=200, activation='sigmoid'))
model10.add(Dense(units=nb_out10, activation='sigmoid'))

# With adam optimizer and a binary crossentropy loss. We will opimize for model accuracy.
model10.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Verify the architecture 
print(model10.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 21, 400)           652800    
                                                                 
 dropout (Dropout)           (None, 21, 400)           0         
                                                                 
 lstm_1 (LSTM)               (None, 21, 200)           480800    
                                                                 
 dropout_1 (Dropout)         (None, 21, 200)           0         
                                                                 
 lstm_2 (LSTM)               (None, 21, 200)           320800    
                                                                 
 dropout_2 (Dropout)         (None, 21, 200)           0         
                                                                 
 lstm_3 (LSTM)               (None, 200)               3

In [12]:
# build the network for failure2 data
# Feature weights
nb_features20 = seq_array20.shape[2]
nb_out20 = label_array20.shape[1]

# LSTM model
model20 = Sequential()

# The first layer
model20.add(LSTM(
         input_shape=(sequence_length20, nb_features20),
         units=400,
         return_sequences=True))

# Plus a 20% dropout rate
model20.add(Dropout(0.2))

# The second layer
model20.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model20.add(Dropout(0.2))

# The third layer
model20.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model20.add(Dropout(0.2))

# The fourth layer
model20.add(LSTM(
          units=200,
          return_sequences=False))

# Plus a 20% dropout rate
model20.add(Dropout(0.2))

# Dense sigmoid layer
model20.add(Dense(units=200, activation='sigmoid'))
model20.add(Dense(units=200, activation='sigmoid'))
model20.add(Dense(units=200, activation='sigmoid'))
model20.add(Dense(units=nb_out20, activation='sigmoid'))

# With adam optimizer and a binary crossentropy loss. We will opimize for model accuracy.
model20.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Verify the architecture 
print(model20.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 21, 400)           652800    
                                                                 
 dropout_4 (Dropout)         (None, 21, 400)           0         
                                                                 
 lstm_5 (LSTM)               (None, 21, 200)           480800    
                                                                 
 dropout_5 (Dropout)         (None, 21, 200)           0         
                                                                 
 lstm_6 (LSTM)               (None, 21, 200)           320800    
                                                                 
 dropout_6 (Dropout)         (None, 21, 200)           0         
                                                                 
 lstm_7 (LSTM)               (None, 200)              

In [13]:
# build the network for failure3 data
# Feature weights
nb_features30 = seq_array30.shape[2]
nb_out30 = label_array30.shape[1]

# LSTM model
model30 = Sequential()

# The first layer
model30.add(LSTM(
         input_shape=(sequence_length30, nb_features30),
         units=400,
         return_sequences=True))

# Plus a 20% dropout rate
model30.add(Dropout(0.2))

# The second layer
model30.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model30.add(Dropout(0.2))

# The third layer
model30.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model30.add(Dropout(0.2))

# The fourth layer
model30.add(LSTM(
          units=200,
          return_sequences=False))

# Plus a 20% dropout rate
model30.add(Dropout(0.2))

# Dense sigmoid layer
model30.add(Dense(units=200, activation='sigmoid'))
model30.add(Dense(units=200, activation='sigmoid'))
model30.add(Dense(units=200, activation='sigmoid'))
model30.add(Dense(units=nb_out30, activation='sigmoid'))

# With adam optimizer and a binary crossentropy loss. We will opimize for model accuracy.
model30.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Verify the architecture 
print(model30.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 21, 400)           652800    
                                                                 
 dropout_8 (Dropout)         (None, 21, 400)           0         
                                                                 
 lstm_9 (LSTM)               (None, 21, 200)           480800    
                                                                 
 dropout_9 (Dropout)         (None, 21, 200)           0         
                                                                 
 lstm_10 (LSTM)              (None, 21, 200)           320800    
                                                                 
 dropout_10 (Dropout)        (None, 21, 200)           0         
                                                                 
 lstm_11 (LSTM)              (None, 200)              

In [14]:
# build the network for failure4 data
# Feature weights
nb_features40 = seq_array40.shape[2]
nb_out40 = label_array40.shape[1]

# LSTM model
model40 = Sequential()

# The first layer
model40.add(LSTM(
         input_shape=(sequence_length40, nb_features40),
         units=400,
         return_sequences=True))

# Plus a 20% dropout rate
model40.add(Dropout(0.2))

# The second layer
model40.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model40.add(Dropout(0.2))

# The third layer
model40.add(LSTM(
          units=200,
          return_sequences=True))

# Plus a 20% dropout rate
model40.add(Dropout(0.2))

# The fourth layer
model40.add(LSTM(
          units=200,
          return_sequences=False))

# Plus a 20% dropout rate
model40.add(Dropout(0.2))

# Dense sigmoid layer
model40.add(Dense(units=200, activation='sigmoid'))
model40.add(Dense(units=200, activation='sigmoid'))
model40.add(Dense(units=200, activation='sigmoid'))
model40.add(Dense(units=nb_out40, activation='sigmoid'))

# With adam optimizer and a binary crossentropy loss. We will opimize for model accuracy.
model40.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Verify the architecture 
print(model40.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 21, 400)           652800    
                                                                 
 dropout_12 (Dropout)        (None, 21, 400)           0         
                                                                 
 lstm_13 (LSTM)              (None, 21, 200)           480800    
                                                                 
 dropout_13 (Dropout)        (None, 21, 200)           0         
                                                                 
 lstm_14 (LSTM)              (None, 21, 200)           320800    
                                                                 
 dropout_14 (Dropout)        (None, 21, 200)           0         
                                                                 
 lstm_15 (LSTM)              (None, 200)              

In [15]:
%%time
# fit the network for failure1 data
model10.fit(seq_array10, # Training features
          label_array10, # Training labels
          epochs=20,   # We'll stop after 10 epochs
          batch_size=1024, # 
          validation_split=0.3, # Use 30% of data to evaluate the loss. (val_loss)
          verbose=1, #
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', # Monitor the validation loss
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=5,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto')])
print("Fitting the network for failure1 data complete.")

model10.save('models/machineRULfailure1Predictor.h5')
print("Model10 for failure1 data saved.")

Epoch 1/20
7/7 [==============================] - 98s 12s/step - loss: 0.7336 - accuracy: 0.5102 - val_loss: 0.6875 - val_accuracy: 0.5542
Epoch 2/20
7/7 [==============================] - 80s 11s/step - loss: 0.7009 - accuracy: 0.4920 - val_loss: 0.6963 - val_accuracy: 0.4458
Epoch 3/20
7/7 [==============================] - 81s 12s/step - loss: 0.6968 - accuracy: 0.5121 - val_loss: 0.6873 - val_accuracy: 0.5542
Epoch 4/20


In [ ]:
%%time
# fit the network for failure2 data
model20.fit(seq_array20, # Training features
          label_array20, # Training labels
          epochs=20,   # We'll stop after 10 epochs
          batch_size=1024, # 
          validation_split=0.3, # Use 30% of data to evaluate the loss. (val_loss)
          verbose=1, #
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', # Monitor the validation loss
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=5,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto')])
print("Fitting the network for failure2 data complete.")

model20.save('models/machineRULfailure2Predictor.h5')
print("Model20 for failure2 saved.")

In [ ]:
%%time
# fit the network for failure3 data
model30.fit(seq_array30, # Training features
          label_array30, # Training labels
          epochs=20,   # We'll stop after 10 epochs
          batch_size=1024, # 
          validation_split=0.3, # Use 30% of data to evaluate the loss. (val_loss)
          verbose=1, #
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', # Monitor the validation loss
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=5,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto')])
print("Fitting the network for failure3 data complete.")

model30.save('models/machineRULfailure3Predictor.h5')
print("Model30 for failure3 saved.")

In [ ]:
%%time
# fit the network for failure4 data
model40.fit(seq_array40, # Training features
          label_array40, # Training labels
          epochs=20,   # We'll stop after 10 epochs
          batch_size=1024, # 
          validation_split=0.3, # Use 30% of data to evaluate the loss. (val_loss)
          verbose=1, #
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', # Monitor the validation loss
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=5,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto')])
print("Fitting the network for failure4 data complete.")

model40.save('models/machineRULfailure4Predictor.h5')
print("Model40 for failure4 saved.")

In [ ]:
model10 = keras.models.load_model('models/machineRULfailure1Predictor.h5')
print("Model10 loaded.")
model20 = keras.models.load_model('models/machineRULfailure2Predictor.h5')
print("Model20 loaded.")
model30 = keras.models.load_model('models/machineRULfailure3Predictor.h5')
print("Model30 loaded.")
model40 = keras.models.load_model('models/machineRULfailure4Predictor.h5')
print("Model40 loaded.")

In [ ]:
# training metrics
# failure1
scores10 = model10.evaluate(seq_array10, label_array10, verbose=1, batch_size=200)
print('Training Accurracy for model10: {}'.format(scores10[1]))

# failure2
scores20 = model20.evaluate(seq_array20, label_array20, verbose=1, batch_size=200)
print('Training Accurracy for model20: {}'.format(scores20[1]))

# failure3
scores30 = model30.evaluate(seq_array30, label_array30, verbose=1, batch_size=200)
print('Training Accurracy for model30: {}'.format(scores30[1]))

# failure4
scores40 = model40.evaluate(seq_array40, label_array40, verbose=1, batch_size=200)
print('Training Accurracy for model40: {}'.format(scores40[1]))


In [ ]:
# make predictions and compute confusion matrix
print('Training Confusion matrices\n- x-axis is true labels.\n- y-axis is predicted labels')

# failure1
print('')
y_pred10 = (model10.predict(seq_array10,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true10 = label_array10
cm10 = confusion_matrix(y_true10, y_pred10)
print('Confusion matrix for model10:')
print(cm10)

# failure2
print('')
y_pred20 = (model20.predict(seq_array20,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true20 = label_array20
cm20 = confusion_matrix(y_true20, y_pred20)
print('Confusion matrix for model20:')
print(cm20)

# failure3
print('')
y_pred30 = (model30.predict(seq_array30,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true30 = label_array30
cm30 = confusion_matrix(y_true30, y_pred30)
print('Confusion matrix for model30:')
print(cm30)

# failure4
print('')
y_pred40 = (model40.predict(seq_array40,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true40 = label_array40
cm40 = confusion_matrix(y_true40, y_pred40)
print('Confusion matrix for model40:')
print(cm40)

In [ ]:
# compute precision and recall
# failure1
precision10 = precision_score(y_true10, y_pred10)
recall10 = recall_score(y_true10, y_pred10)
f110 = 2 * (precision10 * recall10) / (precision10 + recall10)
print('Metrics for model10:', '\n', 'Training Precision: ', precision10, '\n', 'Training Recall: ', recall10, '\n', 'Training F1 Score:', f110, '\n')

# failure2
precision20 = precision_score(y_true20, y_pred20)
recall20 = recall_score(y_true20, y_pred20)
f120 = 2 * (precision20 * recall20) / (precision20 + recall20)
print('Metrics for model20:', '\n', 'Training Precision: ', precision20, '\n', 'Training Recall: ', recall20, '\n', 'Training F1 Score:', f120, '\n')

# failure3
precision30 = precision_score(y_true30, y_pred30)
recall30 = recall_score(y_true30, y_pred30)
f130 = 2 * (precision30 * recall30) / (precision30 + recall30)
print('Metrics for model30:', '\n', 'Training Precision: ', precision30, '\n', 'Training Recall: ', recall30, '\n', 'Training F1 Score:', f130, '\n')

# failure4
precision40 = precision_score(y_true40, y_pred40)
recall40 = recall_score(y_true40, y_pred40)
f140 = 2 * (precision40 * recall40) / (precision40 + recall40)
print('Metrics for model40:', '\n', 'Training Precision: ', precision40, '\n', 'Training Recall: ', recall40, '\n', 'Training F1 Score:', f140, '\n')

## Model testing

In [ ]:
#failure1
seq_array_test_last10 = [test_df10[test_df10['machineID']==id10][sequence_cols10].values[-sequence_length10:] 
                       for id10 in test_df10['machineID'].unique() if len(test_df10[test_df10['machineID']==id10]) >= sequence_length10]
seq_array_test_last10 = np.asarray(seq_array_test_last10).astype(np.float32)
#seq_array_test_last10.shape

#failure2
seq_array_test_last20 = [test_df20[test_df20['machineID']==id20][sequence_cols20].values[-sequence_length20:] 
                       for id20 in test_df20['machineID'].unique() if len(test_df20[test_df20['machineID']==id20]) >= sequence_length20]
seq_array_test_last20 = np.asarray(seq_array_test_last20).astype(np.float32)
#seq_array_test_last20.shape

#failure3
seq_array_test_last30 = [test_df30[test_df30['machineID']==id30][sequence_cols30].values[-sequence_length30:] 
                       for id30 in test_df30['machineID'].unique() if len(test_df30[test_df30['machineID']==id30]) >= sequence_length30]
seq_array_test_last30 = np.asarray(seq_array_test_last30).astype(np.float32)
#seq_array_test_last30.shape

#failure4
seq_array_test_last40 = [test_df40[test_df40['machineID']==id40][sequence_cols40].values[-sequence_length40:] 
                       for id40 in test_df40['machineID'].unique() if len(test_df40[test_df40['machineID']==id40]) >= sequence_length40]
seq_array_test_last40 = np.asarray(seq_array_test_last40).astype(np.float32)
#seq_array_test_last40.shape

In [ ]:
#failure1
y_mask10 = [len(test_df10[test_df10['machineID']==id10]) >= sequence_length10 for id10 in test_df10['machineID'].unique()]
label_array_test_last10 = test_df10.groupby('machineID')['label1'].nth(-1)[y_mask10].values
label_array_test_last10 = label_array_test_last10.reshape(label_array_test_last10.shape[0],1).astype(np.float32)
#print(seq_array_test_last10.shape)
#print(label_array_test_last10.shape)

#failure2
y_mask20 = [len(test_df20[test_df20['machineID']==id20]) >= sequence_length20 for id20 in test_df20['machineID'].unique()]
label_array_test_last20 = test_df20.groupby('machineID')['label1'].nth(-1)[y_mask20].values
label_array_test_last20 = label_array_test_last20.reshape(label_array_test_last20.shape[0],1).astype(np.float32)
#print(seq_array_test_last20.shape)
#print(label_array_test_last20.shape)

#failure3
y_mask30 = [len(test_df30[test_df30['machineID']==id30]) >= sequence_length30 for id30 in test_df30['machineID'].unique()]
label_array_test_last30 = test_df30.groupby('machineID')['label1'].nth(-1)[y_mask30].values
label_array_test_last30 = label_array_test_last30.reshape(label_array_test_last30.shape[0],1).astype(np.float32)
#print(seq_array_test_last30.shape)
#print(label_array_test_last30.shape)

#failure4
y_mask40 = [len(test_df40[test_df40['machineID']==id40]) >= sequence_length40 for id40 in test_df40['machineID'].unique()]
label_array_test_last40 = test_df40.groupby('machineID')['label1'].nth(-1)[y_mask40].values
label_array_test_last40 = label_array_test_last40.reshape(label_array_test_last40.shape[0],1).astype(np.float32)
#print(seq_array_test_last40.shape)
#print(label_array_test_last40.shape)

In [ ]:
# test metrics
#failure1
scores_test10 = model10.evaluate(seq_array_test_last10, label_array_test_last10, verbose=2)
print('Test Accurracy - failure1: {}'.format(scores_test10[1]))

#failure2
scores_test20 = model20.evaluate(seq_array_test_last20, label_array_test_last20, verbose=2)
print('Test Accurracy - failure2: {}'.format(scores_test20[1]))

#failure3
scores_test30 = model30.evaluate(seq_array_test_last30, label_array_test_last30, verbose=2)
print('Test Accurracy - failure3: {}'.format(scores_test30[1]))

#failure4
scores_test40 = model40.evaluate(seq_array_test_last40, label_array_test_last40, verbose=2)
print('Test Accurracy - failure4: {}'.format(scores_test40[1]))

In [ ]:
# make predictions and compute confusion matrix
#failure1
y_pred_test10 = (model10.predict(seq_array_test_last10,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true_test10 = label_array_test_last10
print('Confusion matrix - failure1\n- x-axis is true labels.\n- y-axis is predicted labels')
cm10 = confusion_matrix(y_true_test10, y_pred_test10)
print(cm10)

#failure2
y_pred_test20 = (model20.predict(seq_array_test_last20,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true_test20 = label_array_test_last20
print('Confusion matrix - failure2\n- x-axis is true labels.\n- y-axis is predicted labels')
cm20 = confusion_matrix(y_true_test20, y_pred_test20)
print(cm20)

#failure3
y_pred_test30 = (model30.predict(seq_array_test_last30,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true_test30 = label_array_test_last30
print('Confusion matrix - failure3\n- x-axis is true labels.\n- y-axis is predicted labels')
cm30 = confusion_matrix(y_true_test30, y_pred_test30)
print(cm30)

#failure4
y_pred_test40 = (model40.predict(seq_array_test_last40,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true_test40 = label_array_test_last40
print('Confusion matrix - failure4\n- x-axis is true labels.\n- y-axis is predicted labels')
cm40 = confusion_matrix(y_true_test40, y_pred_test40)
print(cm40)

In [ ]:
# compute precision and recall
#failure1
precision_test10 = precision_score(y_true_test10, y_pred_test10)
recall_test10 = recall_score(y_true_test10, y_pred_test10)
f1_test10 = 2 * (precision_test10 * recall_test10) / (precision_test10 + recall_test10)
print('Failure1:', '\n', 'Test Precision: ', precision_test10, '\n', 'Test Recall: ', recall_test10, '\n', 'Test F1 Score:', f1_test10)

#failure2
precision_test20 = precision_score(y_true_test20, y_pred_test20)
recall_test20 = recall_score(y_true_test20, y_pred_test20)
f1_test20 = 2 * (precision_test20 * recall_test20) / (precision_test20 + recall_test20)
print('Failure2:', '\n', 'Test Precision: ', precision_test20, '\n', 'Test Recall: ', recall_test20, '\n', 'Test F1 Score:', f1_test20)

#failure3
precision_test30 = precision_score(y_true_test30, y_pred_test30)
recall_test30 = recall_score(y_true_test30, y_pred_test30)
f1_test30 = 2 * (precision_test30 * recall_test30) / (precision_test30 + recall_test30)
print('Failure3:', '\n', 'Test Precision: ', precision_test30, '\n', 'Test Recall: ', recall_test30, '\n', 'Test F1 Score:', f1_test30)

#failure4
precision_test40 = precision_score(y_true_test40, y_pred_test40)
recall_test40 = recall_score(y_true_test40, y_pred_test40)
f1_test40 = 2 * (precision_test40 * recall_test40) / (precision_test40 + recall_test40)
print('Failure4:', '\n', 'Test Precision: ', precision_test40, '\n', 'Test Recall: ', recall_test40, '\n', 'Test F1 Score:', f1_test40)